In [3]:
pip install langchain==0.0.55 requests openai transformers faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
import requests

In [6]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [7]:
def get_wiki_data(title, first_paragraph_only):
    url = f"https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&explaintext=1&titles={title}"
    if first_paragraph_only:
        url += "&exintro=1"
    data = requests.get(url).json()
    return Document(
        page_content=list(data["query"]["pages"].values())[0]["extract"],
        metadata={"source": f"https://en.wikipedia.org/wiki/{title}"},
    )

In [8]:
from langchain.text_splitter import CharacterTextSplitter

In [9]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0))

def print_answer(question):
    print(
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=4),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]
    )

In [10]:
import pathlib
import subprocess
import tempfile

In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

In [16]:
pip install faiss

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.


In [12]:
search_index = FAISS.from_documents(sources, OpenAIEmbeddings())

In [13]:
def get_github_docs(repo_owner, repo_name):
    with tempfile.TemporaryDirectory() as d:
        subprocess.check_call(
            f"git clone --depth 1 https://github.com/{repo_owner}/{repo_name}.git .",
            cwd=d,
            shell=True,
        )
        git_sha = (
            subprocess.check_output("git rev-parse HEAD", shell=True, cwd=d)
            .decode("utf-8")
            .strip()
        )
        repo_path = pathlib.Path(d)
        markdown_files = list(repo_path.glob("*/*.md")) + list(
            repo_path.glob("*/*.mdx")
        )
        for markdown_file in markdown_files:
            with open(markdown_file, "r") as f:
                relative_path = markdown_file.relative_to(repo_path)
                github_url = f"https://github.com/{repo_owner}/{repo_name}/blob/{git_sha}/{relative_path}"
                yield Document(page_content=f.read(), metadata={"source": github_url})

In [14]:
sources = get_github_docs("hyperledger", "aries-cloudagent-python")

In [15]:
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)
for source in sources:
    for chunk in splitter.split_text(source.page_content):
        source_chunks.append(Document(page_content=chunk, metadata=source.metadata))

search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings())

Cloning into '.'...


In [16]:
print_answer("what is a hyperledger aries?")

 Hyperledger Aries is an open source, secure, and privacy-preserving framework for decentralized identity. It is designed to help developers create applications that interact with decentralized identity systems.
SOURCES: https://www.hyperledger.org/projects/hyperledger-aries


In [17]:
print_answer("Cna you please describe how the multitenancy model works?")

 The multitenancy model allows multiple tenants to use the same instance of an application. It is a way of partitioning a single instance of software so that multiple tenants can use it simultaneously. Each tenant has its own data and configuration settings, and the application is designed to isolate tenants from each other.
SOURCES: https://github.com/hyperledger/aries-cloudagent-python/blob/master/Multitenancy.md


In [18]:
print_answer("Can you please describe the aries architecture?")

 Aries architecture is a cloud-based agent-oriented architecture that enables secure, decentralized communication between agents. It is based on Hyperledger Indy, a distributed ledger technology, and uses the Aries Cloud Agent Python (ACA-Py) to implement an application. It includes features such as exchanging messages, issuing and proving credentials, and running a local Indy ledger.
SOURCES:
https://github.com/hyperledger/aries-cloudagent-python/blob/45c832658245747a3366735f6179362d127bae02/docs/README.md
https://github.com/hyperledger/aries-cloudagent-python/blob/45c832658245747a3366735f6179362d127bae02/demo/README.md
